<font color=#0099ff size=5 face="載入所需模組"> 載入所需模組</font>

In [6]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<font color=#0099ff size=5 face="載入所需模組"> 將 train，test載入</font>

In [7]:
train = pd.read_csv("C:\\pythonwork\\hw1\\train.csv")
test = pd.read_csv("C:\\pythonwork\\hw1\\public_testing.csv")

<font color=#0099ff size=5 face="載入所需模組"> 數據清洗</font>

In [8]:
#將train&test data做合併
data = train.append(test)
data.reset_index(inplace= True,drop=True)

#將列(data_columns)重新命名
data = data.rename(columns={'educational-num': 'educationalNum'})
data = data.rename(columns={'capital-gain': 'capitalGain'})
data = data.rename(columns={'capital-loss': 'capitalLoss'})
data = data.rename(columns={'hours-per-week': 'hoursPerWeek'})

#選取所需特徵
#這五項影響力最大
cols=['age','capitalGain','capitalLoss','hoursPerWeek','educationalNum']
data_pred = data[cols]
#將income變成1,0
#'>50K':1,'<=50K':0
data['income'] = data['income'].map({'>50K':1,'<=50K':0})
y=data['income']

<font color=#0099ff size=5 face="載入所需模組"> 特徵欄位進行標準化</font>

In [9]:
#使用 preprocessing.MinMaxScaler 進行標準化
#輸入參數 feature_range 設定標準化之後的範圍(0,1)
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))

#使用 minmax_scale.fit_transform 輸入參數data_pred(特徵欄位) 進行標準化
ScaledFeatures = minmax_scale.fit_transform(data_pred)

#查看特徵標準化後，前2筆資料
ScaledFeatures[:2]

array([[ 0.06849315,  0.        ,  0.        ,  0.15957447,  0.57142857],
       [ 0.30136986,  0.        ,  0.        ,  0.37234043,  0.85714286]])

<font color=#0099ff size=5 face="載入所需模組"> 使用TPOT</font>

In [10]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(data_pred,y, train_size=0.8, test_size=0.2)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, n_jobs=-1)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


Generation 1 - Current best internal CV score: 0.7564333073918086


Generation 2 - Current best internal CV score: 0.759534082585607


Generation 3 - Current best internal CV score: 0.759534082585607


Generation 4 - Current best internal CV score: 0.759534082585607


Generation 5 - Current best internal CV score: 0.759534082585607



Best pipeline: KNeighborsClassifier(RandomForestClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.1, min_samples_leaf=20, min_samples_split=18, n_estimators=100), n_neighbors=66, p=2, weights=distance)
0.7625


<font color=#0099ff size=5 face="載入所需模組"> 輸出TPOT成py檔</font>

In [11]:
tpot.export("tpot.py")

True

<font color=#0099ff size=5 face="載入所需模組"> 使用 LinearSVC 進行預測</font>

In [31]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator

features = data_pred
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, data['income'].values, random_state=42)

# Score on the training set was:0.7649454823251615
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GradientBoostingClassifier(learning_rate=0.01, max_depth=2, max_features=0.8, min_samples_leaf=18, min_samples_split=2, n_estimators=100, subsample=1.0)),
    LinearSVC(C=10.0, dual=False, loss="squared_hinge", penalty="l2", tol=1e-05)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
exported_pipeline.score(testing_features,testing_target)


0.78500000000000003

<font color=#0099ff size=5 face="載入所需模組"> 將預測結果儲存成submission.py</font>

In [33]:
# Create the submission file
final = pd.DataFrame({'fnlwt': test['fnlwgt'], 'result': results})
final.to_csv('C:\\pythonwork\\hw1\\submission.csv', index = False)